In [33]:
import requests
import pandas as pd
import gzip
import io
import shutil
import os

## Define date and url


In [40]:
index = 'BTCUSD'
type = 'spot_index'
base_url = 'https://public.bybit.com/'
date = '2019-10-01'
data_extension = '.csv.gz'
file_name = index + date + '_index_price' + data_extension
url = base_url + type + '/' + index + '/'  + file_name

In [41]:
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-01_index_price.csv.gz

'https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-01_index_price.csv.gz'

## Get list of dates from the data source page

In [57]:
url_dates = base_url + type + '/' + index + '/'

In [112]:
r = requests.get(url_dates, allow_redirects=True)
dates_content = r.text
dates_content = dates_content.split('<li>')
dates_content = [row.split('"')[1] for row in dates_content]
dates_content = dates_content[1:]
dates = [row.split('BTCUSD')[1].split('_index')[0] for row in dates_content]

In [120]:
df_dates = pd.DataFrame()
df_dates['date'] = dates
df_dates['date'] = pd.to_datetime(df_dates['date'])
df_dates['exists'] = 1

### Visualize the window the data is available

In [126]:
import plotly.express as px

df = px.data.stocks()
fig = px.scatter(df_dates, x='date', y='exists')
fig.show()

## Get content from url

In [46]:
def get_content(url):
    r = requests.get(url, allow_redirects=True)
    content = r.content
    return content

## Write the content into defined path

In [53]:
compressed_path = 'data/compressed/'
extracted_path = 'data/extracted/'

In [127]:
def write_content(compressed_path, content, file_name):
    open(os.path.join(compressed_path,file_name), 'wb').write(content)

### Read the file in csv form

In [128]:
def read_content_from_csv(compressed_path, extracted_path, file_name):
    with gzip.open(os.path.join(compressed_path,file_name), 'rb') as f_in:
        file_name = file_name.replace('.gz','')
        with open(os.path.join(extracted_path,file_name), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

## Download and write content for each date

In [139]:
def download_all_dates(*args):
    for date in dates:
        file_name = index + date + '_index_price' + data_extension
        url = base_url + type + '/' + index + '/'  + file_name
        get_content(url)
        write_content(compressed_path, content, file_name)

In [142]:
args = {'dates' : dates,
        'index' : index,
        'data_extension' : data_extension,
        'base_url' : base_url,
        'file_name' : file_name,
         ''}
download_all_dates(args)

https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-01_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-02_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-03_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-04_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-05_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-06_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-07_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-08_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-09_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-10_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-11_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-12_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD201

https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-01_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-02_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-03_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-04_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-05_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-06_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-07_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-08_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-09_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-10_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-11_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-12_index_price.csv.gz
https://public.bybit.com/spot_index/BTCUSD/BTCUSD201